# Библиотеки

In [5]:
from zlib import crc32

import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from statsmodels.formula.api import ols
import statsmodels.api as sm

# Код генерации выборки

In [2]:
import re
EMAIL_REGEX = re.compile(r"[^@]+@phystech.edu")

def generate_dataset(code):
    rs = np.random.RandomState(code)
    tip = rs.randint(low = 0, high = 4)
    C1 = rs.choice([1, 5], size = 1000)
    C2 = rs.choice([2, 4], size = 1000)
    if tip == 0 :
        data = rs.randn(1000)
    if tip == 1:
        data = []
        for c1,c2 in zip(C1, C2):
            if c1 == 1:
                data.append(rs.randn(1) + 1)
            if c1 == 5:
                data.append(rs.randn(1) - 1)
        data = np.array(data)
    if tip == 2:
        data = []
        for c1,c2 in zip(C1, C2):
            if c1==1:
                c1 = 0
            else:
                c1= 1
            if c2 ==2:
                c2 = 0
            else:
                c2 = 1
            data.append(rs.randn(1) + c1 + c2)
    if tip == 3:# on c1, c2
        data = []
        for c1,c2 in zip(C1,C2):
            if c1==1:
                c1 = 0
            else:
                c1= 1
            if c2 ==2:
                c2 = 0
            else:
                c2 = 1
            data.append(rs.randn(1) + c1 + c2 + 5*c1*c2)
        data = np.array(data)
    return np.hstack([np.array(data).reshape(1000, 1), C1.reshape(1000, 1), C2.reshape(1000, 1)])

# Генерация выборки для вашей почты

<span style="color:red">
    ВАЖНО!
    Почта, которую укажете ниже и почта с которой Вы отправляете домашнее задание должна совпадать!
    В момент проверки задания алгоритм генерирует выборку на основе почты из анкеты!
</span>

Внимательно проверьте почту для которой выполняется задание!

In [3]:
task = dict()
task['mail'] = input(prompt='Enter your mail: ')
assert EMAIL_REGEX.match(task['mail']), 'Not a mail in the phystech.edu domain'
task['id'] = crc32(task['mail'].encode('utf-8'))
task['data'] = generate_dataset(task['id'])

task

Enter your mail: egorov.sa@phystech.edu


{'mail': 'egorov.sa@phystech.edu',
 'id': 1581189463,
 'data': array([[ 0.28568997,  1.        ,  2.        ],
        [-1.44316383,  5.        ,  2.        ],
        [ 0.36034202,  1.        ,  2.        ],
        ...,
        [ 2.16191444,  1.        ,  2.        ],
        [ 1.08130347,  5.        ,  2.        ],
        [ 1.14789086,  1.        ,  4.        ]])}

# Работа с выборкой

In [10]:
data = pd.DataFrame(task['data'], columns=['f', 'c1', 'c2'])
data.describe()

,f,c1,c2
count,1000.000000,1000.000000,1000.000000
mean,-0.019024,3.072000,3.024000
std,1.401674,1.999704,1.000212
min,-4.179774,1.000000,2.000000
25%,-1.019398,1.000000,2.000000
50%,-0.040992,5.000000,4.000000
75%,1.063825,5.000000,4.000000
max,3.775650,5.000000,4.000000


- Проверить с использованием двухфакторного дисперсионного анализа, влияет ли переменная C1 на переменную f в данной линейной модели?

- Проверить с использованием двухфакторного дисперсионного анализа, влияет ли переменная C2 на переменную f в данной линейной модели?

- Проверить с использованием двухфакторного дисперсионного анализа, влияет ли межфакторное взаимодействие на переменную f в данной линейной модели?

In [11]:
lm = ols('f ~  c1 + c2 + c1:c2',  data=data).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                      f   R-squared:                       0.496
Model:                            OLS   Adj. R-squared:                  0.494
Method:                 Least Squares   F-statistic:                     326.7
Date:                Mon, 23 Oct 2023   Prob (F-statistic):          1.16e-147
Time:                        14:20:42   Log-Likelihood:                -1413.5
No. Observations:                1000   AIC:                             2835.
Df Residuals:                     996   BIC:                             2855.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.5799      0.185      8.547      0.0

In [12]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      f   R-squared:                       0.496
Model:                            OLS   Adj. R-squared:                  0.494
Method:                 Least Squares   F-statistic:                     326.7
Date:                Mon, 23 Oct 2023   Prob (F-statistic):          1.16e-147
Time:                        14:20:44   Log-Likelihood:                -1413.5
No. Observations:                1000   AIC:                             2835.
Df Residuals:                     996   BIC:                             2855.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.5799      0.185      8.547      0.000       1.217       1.943
c1            -0.5451      0.050    -10.848      0.000      -0.644      -0.447
c2            -0.0277      0.058     -0.480      0.632      -0.141       0.086
c1:c2          0.0172      0.016      1.092      0.275      -0.014       0.048
==============================================================================
Omnibus:                        1.507   Durbin-Watson:                   1.930
Prob(Omnibus):                  0.471   Jarque-Bera (JB):                1.506
Skew:                          -0.095   Prob(JB):                        0.471
Kurtosis:                       2.981   Cond. No.                         78.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
lm.f_test("c1 = c2 = c1:c2 = 0")

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=326.7188354940051, p=1.157977215996802e-147, df_denom=996, df_num=3>

Итого:

1. Переменная `с1` влияет на переменную `f` в линейной модели (`p-value = 0.000`)

2. Переменная `с1` не влияет на переменную `f` в линейной модели (`p-value = 0.632`)

3. Межжфакторное взаимодействие не влияет на переменную `f` в линейной модели (`p-value = 0.275`)